In [13]:
import gurobipy as grb
from gurobipy import GRB
import numpy as np

#定义主问题，变量x为按该方案剪裁多少张
#column是从子问题中生成的剪裁方案矩阵,cost是某方案的成本矩阵
#求当前所有方案的cost最小值并返回dual作为子问题的参数，如果已经是最优，则停止并返回目标函数值和变量x
def master_problem(column, vtype):
    m = grb.Model()
    x = m.addMVar(shape=column.shape[1], lb=0, vtype=vtype)
    m.addConstr(lhs=column @ x >= demand_number_array)
    print(x)
    m.setObjective(cost@x, GRB.MINIMIZE)
    m.optimize()

    if vtype == GRB.CONTINUOUS:
        return np.array(m.getAttr('Pi', m.getConstrs()))
    else:
        return m.objVal, np.array(m.getAttr('X'))


def restricted_lp_master_problem(column):
    return master_problem(column, GRB.CONTINUOUS)


def restricted_ip_master_problem(column):
    return master_problem(column, GRB.INTEGER)








<(3,) matrix variable>
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 3 rows, 3 columns and 3 nonzeros
Model fingerprint: 0xc8471849
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [5e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+01]
Presolve removed 3 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.5500000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  6.550000000e+02
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0xd3368549
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e+00, 7e+00]
  Objective range  [3e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+00, 9e+00]
Found heuristic solution

In [ ]:
#定义子问题，x为该种剪裁方案剪裁多少Demand1\Demand2\Demand3,限定为整数
#由于使用三种布料，所以用三个cost分别求解，求出-reduced cost
#-reduced cost小于0证明原问题非最优，找到最小值并将其对应的x作为新方案列添加到原问题的方案矩阵中
def knapsack_subproblem(kk):
    global cost,roll_width_use
    objc=[]
    xc=[]
    
    for i in range(len(roll_width)):
        m = grb.Model()
        x = m.addMVar(shape=kk.shape[0], lb=0, vtype=GRB.INTEGER)
        m.addConstr(lhs=demand_width_array @ x <= roll_width[i])
        m.setObjective(cost[i] - kk @ x, GRB.MINIMIZE)
        m.optimize()
        objc.append(m.objVal)
        xc.append(m.getAttr('X'))
#寻找最小的-reduced cost
    obj=min(objc)
    objind=np.argmin(objc)
    
#只要最小的-reduced_cost<0，非最优，生成新列
    flag_new_column = obj < 0
    
    if flag_new_column:
        new_column = xc[objind]
        cost=np.append(cost,[cost[objind]])
        roll_width_use=np.append(roll_width_use,[roll_width[objind]])
        
#原问题最优，不生成新列
    else:
        new_column = None

    return flag_new_column, new_column

In [ ]:
#定义参数和初始cut方案
cost=np.array([5,9,10])
roll_width = np.array([9,14,16])
demand_width_array = np.array([4,5,7])
demand_number_array = np.array([30,20,40])
initial_cut_pattern = np.diag(np.floor(roll_width[0]/ demand_width_array))
roll_width_use = np.array([9,9,9])

#flag_new_cut_pattern为True证明非最优，循环迭代求解主问题和子问题，直到所有-reduced cost<0
flag_new_cut_pattern = True
new_cut_pattern = None
cut_pattern = initial_cut_pattern
while flag_new_cut_pattern:
    if new_cut_pattern:
        cut_pattern = np.column_stack((cut_pattern, new_cut_pattern))
    kk = restricted_lp_master_problem(cut_pattern)
    flag_new_cut_pattern, new_cut_pattern = knapsack_subproblem(kk)

minimal_cost, optimal_number = restricted_ip_master_problem(cut_pattern)

In [ ]:
print('************************************************')
print('parameter:')
print(f'roll_width: {roll_width}')
print(f'demand_width_array: {demand_width_array}')
print(f'demand_number_array: {demand_number_array}')

print('result:')
print(f'minimal_cost: {minimal_cost}')
for op in range(len(optimal_number)):
    if optimal_number[op]!=0:
        print(f'cut_pattern: {cut_pattern[:,op]}')
        print(f'roll_type:{roll_width_use[op]}')
        print(f'pattern_number: {optimal_number[op]}')

#最小成本为305
#模式1：将长度为9的布料裁剪成4+4，模式2：将长度为9的布料裁剪成4+5，模式3：将长度为14的布料裁剪成7+7
#为满足需求，裁剪模式1 5张，裁剪模式2 20张， 裁剪模式3 20张
